In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine, text


In [4]:
huddf = pd.read_csv('data/hud_insured_multifamily.csv')

C:\Users\mdalt\AppData\Local\Temp\ipykernel_15760\3926192688.py:1: DtypeWarning: Columns (2,19,22,35,36,41,89,90,91,92,95,103,109,111,121,123,125,126,127,164,169,170,176,205,209,217,274) have mixed types. Specify dtype option on import or set low_memory=False.
  huddf = pd.read_csv('data/hud_insured_multifamily.csv')


In [5]:
huddf.head

<bound method NDFrame.head of        OBJECTID  PROPERTY_ID      UGLG_KEY HUB_NAME_TEXT  \
0           1.0  800225589.0       3457600           OHP   
1           2.0  800225933.0       4878544  Headquarters   
2           3.0  800225983.0       1711202           OHP   
3           4.0  800226573.0       5327855           OHP   
4           5.0  800226611.0       1379388           OHP   
...         ...          ...           ...           ...   
16519   16519.0  800254374.0  3401343800.0      New York   
16520   16520.0  800255132.0     4829000.0    Fort Worth   
16521   16521.0  800255134.0     1963525.0   Kansas City   
16522   16522.0  800255144.0       24035.0     Baltimore   
16523   16523.0  800255510.0     2674960.0       Detroit   

      SERVICING_SITE_NAME_TEXT PROJECT_MANAGER_NAME_TEXT  \
0                          OHP                       NaN   
1          Headquarters - BRSC          LESLEY N JOHNSON   
2                          OHP                       NaN   
3        

In [7]:
pip install langchain


Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install langchain-pinecone

   ---------------------------------------- 0.0/215.9 kB ? eta -:--:--
   ----------- ---------------------------- 61.4/215.9 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 215.9/215.9 kB 2.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


“Can an LLM answer the questions? How does it do with easy stuff, how does it do with harder stuff, how easy is it to break (e.g., give me 10 nursing homes that start with the letter A), how does performance improve given a change in the input style / format” 

## give me 10 nursing homes that start with the letter A)